# '''Hobbit viajero'''
Bilbo Baggins emprenderá un viaje fuera de la Comarca. Ya que pasará dos días de camino por el bosque antes de llegar a algún pueblo, por lo que deberá cargar con las provisiones suficientes.

Deseamos cargar con provisiones suficientes para no morir de hambre, pero Bilbo es un Hobbit ocurrente, por lo que considera cargar con sus libretas de poemas y demás. Debe encontrar la combinación adecuada para que el viaje sea soportable.

In [13]:
#Tiene dos mochilas que pueden contener cuando mucho 5.0L y 3.8, respectivamente:
bags = range(1,3)
max_volume_bag_1 = 50000
max_volume_bag_2 = 35000
#Pero tampoco queremos lastimar nuestras espaldas, pues no habrá elfos de los que curan:
max_mass = 35000
#Los hobbits comen siete veces al día.
#Pero Bilbo está dispuesto a comer quizá sólo 5 veces.
#Hay dos tipos de nutrientes esenciales: nutriente 1 y nutriente 2. Diario debe ingerir al menos:
daily_minimum_nutrient_1 = 1000
daily_minimum_nutrient_2 = 3000

In [14]:
#Estas son todas las provisiones posibles:
provisiones ={
    'Libreta de poemas': {'volumen': 200, 'masa': 200, 'valor':700, 'nutricional_1': 1, 'nutricional_2': 1},
    'Libreta de cuentos': {'volumen': 250, 'masa': 250, 'valor': 500, 'nutricional_1': 1, 'nutricional_2': 1},
    'Libreta de canciones': {'volumen': 200, 'masa': 200, 'valor': 100, 'nutricional_1': 0, 'nutricional_2': 0},
    'Cobija muy caliente': {'volumen': 12000, 'masa': 40000, 'valor': 1200, 'nutricional_1': 0, 'nutricional_2': 0},
    'Cobija regular': {'volumen': 12000, 'masa': 24000, 'valor': 400, 'nutricional_1': 0, 'nutricional_2': 0},
    'Pieza de pan de lembas': {'volumen': 100, 'masa': 300, 'valor': 100, 'nutricional_1': 120, 'nutricional_2': 10},
    'Puño de raíces de enano': {'volumen': 50, 'masa': 30, 'valor': 10, 'nutricional_1':0, 'nutricional_2': 40},
    'Pastel de miel': {'volumen': 250, 'masa': 750, 'valor': 250, 'nutricional_1':300, 'nutricional_2': 80},
    'Puño de hojas para la pipa': {'volumen': 50, 'masa': 100, 'valor': 600, 'nutricional_1': 0, 'nutricional_2': 10},
}

In [15]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()
solver = cp_model.CpSolver()

In [16]:
how_many = {}
for item in provisiones:
    for bag in bags:
        if 'Libreta' in item or 'Cobija' in item:
            how_many[(item, bag)] = model.NewBoolVar('%s_bag_%i' %(item, bag))
        else:
            how_many[(item,bag)] = model.NewIntVar(0,100,'units_of_%s_in_bag_%i' %(item, bag))

In [17]:
#No podemos exceder el inventario disponible de cada producto:
for item in provisiones:
    model.Add(sum(how_many[(item, bag)] for bag in bags) <= 100)

In [18]:
model.Add(sum([how_many[(a, i)] for a in ['Cobija muy caliente', 'Cobija regular'] for i in bags]) == 1)
model.Add(sum([how_many[(a, i)] for a in provisiones if 'Libreta' in a for i in bags])>=1)

In [19]:
#No podemos exceder el volumen de las mochilas:
model.Add(sum(provisiones[a]['volumen']*how_many[(a,1)] for a in provisiones) <= max_volume_bag_1)
model.Add(sum(provisiones[a]['volumen']*how_many[(a,2)] for a in provisiones) <= max_volume_bag_2)
#No podemos exceder la masa que puede cargar Bilbo:
model.Add(sum(provisiones[a]['volumen']*how_many[(a,i)] for a in provisiones for i in bags) <= max_mass)
#Bilbo no debe morir de inanición:
model.Add(sum(provisiones[a]['nutricional_1']*how_many[(a,i)] for a in provisiones for i in bags) >= 2*daily_minimum_nutrient_1)
model.Add(sum(provisiones[a]['nutricional_2']*how_many[(a,i)] for a in provisiones for i in bags) >= 2*daily_minimum_nutrient_2)

In [20]:
#Definimos un valor objetivo:
model.Maximize(sum(provisiones[item]['valor']*how_many[(item,bag)] for item in provisiones for bag in bags))

In [21]:
status = solver.Solve(model)

In [22]:
if status in [2,4]:
    for bag in bags:
        print('Carga de mochila %i:' %bag)
        for item in provisiones:
            if solver.Value(how_many[(item,bag)]) == 0:
                continue
            print('%i unidades de %s' %(solver.Value(how_many[(item,bag)]), item))
        print()

Carga de mochila 1:
1 unidades de Libreta de poemas
60 unidades de Pastel de miel
21 unidades de Pieza de pan de lembas
1 unidades de Puño de hojas para la pipa
1 unidades de Libreta de cuentos

Carga de mochila 2:
1 unidades de Cobija muy caliente
1 unidades de Libreta de poemas
99 unidades de Puño de hojas para la pipa
1 unidades de Libreta de cuentos



In [23]:
if status == 4:
    print('Valor de felicidad: %i' %solver.ObjectiveValue())

Valor de felicidad: 80700
